<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [9]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [10]:
%sql sqlite:///movies.sqlite

'Connected: @movies.sqlite'

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [11]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [16]:
%%sql
SELECT DISTINCT title
FROM movies

 * sqlite:///movies.sqlite
Done.


title
Emma
AVATAR2
Titanic


+ Show the performance dates for one of the movies.

In [31]:
%%sql
SELECT title, date
FROM movies
JOIN screenings
USING (imdb_key)

 * sqlite:///movies.sqlite
Done.


title,date
Emma,2021-02-09
Titanic,1997-12-01


+ Show all data concerning performances at a given theatere
  on a given date.

In [23]:
%%sql
SELECT*
FROM screenings
WHERE t_name IS 'Filmstaden' AND date IS '2021-02-09'

 * sqlite:///movies.sqlite
Done.


screen_id,seats,t_name,imdb_key,date,start
4d24c447a7df1154ac7d57db224495d9,112,Filmstaden,ab1234567,2021-02-09,08:00


+ List all customers

In [20]:
%%sql
SELECT username, name
FROM customers

 * sqlite:///movies.sqlite
Done.


username,name
xXAndersXx1337,Anders
360noscope,Pirre
SpaceMan,Elon Musk


+ List all tickets

In [21]:
%%sql
SELECT*
FROM tickets

 * sqlite:///movies.sqlite
Done.


ticket_id,username,screen_id
1604e7a106d95d8d2c11420ad042c85d,SpaceMan,4d24c447a7df1154ac7d57db224495d9


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [25]:
%%sql
INSERT
INTO tickets(username, screen_id)
VALUES  ('xXAndersXx1337', '4d24c447a7df1154ac7d57db224495d9'),
        ('360noscope', '52ff1bc83c860fc93c57d9b811499842');
    


 * sqlite:///movies.sqlite
2 rows affected.


[]

In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2021), we can't do that, instead
  we can use the following idea: each row in a SQLite3 table
  has a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [28]:
%%sql
SELECT ticket_id
FROM tickets
WHERE rowid = last_insert_rowid();

 * sqlite:///movies.sqlite
Done.


ticket_id
78bcbf6a01294efaceb0668bf92c9ba7


+ Try to insert two movie theaters with the same name (this
  should fail).

In [29]:
%%sql
INSERT
INTO theaters(t_name, capacity)
VALUES  ('Royal', 123),
        ('Royal', 321);

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.t_name
[SQL: INSERT
INTO theaters(t_name, capacity)
VALUES  ('Royal', 123),
        ('Royal', 321);]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [33]:
%%sql
INSERT
INTO screenings(seats, t_name, imdb_key, date, start)
VALUES (123, 'Bergakungen', 'ab1234567', '2020-01-01', '23:59');

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT
INTO screenings(seats, t_name, imdb_key, date, start)
VALUES (123, 'Bergakungen', 'ab1234567', '2020-01-01', '23:59');]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

+ Create a ticket where either the user or the performance
  doesnâ€™t exist.

In [38]:
%%sql
INSERT
INTO tickets(username, screen_id)
VALUES ('Parre', '52ff1bc83c860fc93c57d9b811499842');

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT
INTO tickets(username, screen_id)
VALUES ('Parre', '52ff1bc83c860fc93c57d9b811499842');]
(Background on this error at: http://sqlalche.me/e/13/gkpj)